<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0522/0522_TF_IDF_(%EC%8B%A4%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:7 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
g++ is already the newest version (4:9.3.0-1ubuntu2).
python3-dev is already the newest version (3.8.2-0ubuntu2).
python-d

In [3]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [68]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from konlpy.tag import Kkma

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [82]:
df = pd.read_csv("/content/train_for_korean.csv", encoding='utf-8')

In [83]:
df = df.dropna()
df["document"].isnull().sum()

0

In [84]:
df = df.iloc[:10000]
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 10020
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        10000 non-null  int64 
 1   document  10000 non-null  object
 2   label     10000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 312.5+ KB


In [27]:
df.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터 보고 초딩 영화 줄 오버 연기조차 가볍지 않구나,1
2,10265843,너무 재밓었다 그래서 보는 것을 추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다 평점 조정,0
4,6483659,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...,1


In [85]:
df = df.loc[:, ['label','document']]

In [29]:
df.head()

,label,document
0,0,아 더빙 진짜 짜증나네요 목소리
1,1,흠포스터 보고 초딩 영화 줄 오버 연기조차 가볍지 않구나
2,0,너무 재밓었다 그래서 보는 것을 추천한다
3,0,교도소 이야기구먼 솔직히 재미는 없다 평점 조정
4,1,사이몬 페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어 보이기만 했던 커스...


In [9]:
df["document"].isnull().sum()

350

In [86]:
df = df.dropna()
df["document"].isnull().sum()

0

In [87]:
df = df.drop_duplicates()

In [88]:
m = Kkma()
def tokenizer(text):
    return m.morphs(text)

In [13]:
print(tokenizer("오늘도 힘냅시다"))

['오늘', '도', '힘내', 'ㅂ시다']


In [53]:
tfidf = TfidfVectorizer(tokenizer = tokenizer, max_features=10000)
tfidf_matrix = tfidf.fit_transform(df['document'])
y = df['label']
print(tfidf_matrix.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(9889, 10000)


In [33]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [34]:
print(cosine_sim.shape)

(9889, 9889)


In [38]:
cosine_sim_int = cosine_sim.astype(int)

In [89]:
tf_idf = TfidfVectorizer(tokenizer = tokenizer, max_features=1000)
tf_idf_matrix = tf_idf.fit_transform(df['document'])
y = df['label']
print(tf_idf_matrix.shape)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


(9889, 1000)


In [101]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, y, test_size = 0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7911, 1000), (1978, 1000), (7911,), (1978,))

In [56]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
import numpy as np

In [57]:
from sklearn.metrics import mean_squared_error 

models = []
models.append(('Linear', LinearRegression())) # 선형회귀
models.append(('Lasso', Lasso()))
models.append(('Ridge', Ridge()))
models.append(('KNN', KNeighborsRegressor()))
models.append(('SVR', SVR()))
models.append(('RF', RandomForestRegressor()))
models.append(('XGB', XGBRegressor()))



for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    

    msg1 = "%s - train_R2 : %f, test_R2 : %f" % (name, model.score(X_train, y_train), model.score(X_test, y_test))
    print(msg1, end = ' ')

    msg2 = " train_MSE : %f, test MSE : %f" % (np.sqrt(mean_squared_error(y_train, y_pred)), np.sqrt(mean_squared_error(y_test, y_pred_test)))
    print(msg2)

Linear - train_R2 : 0.930655, test_R2 : -2.700594  train_MSE : 0.131667, test MSE : 0.961136
Lasso - train_R2 : 0.000000, test_R2 : -0.001567  train_MSE : 0.500000, test MSE : 0.500022
Ridge - train_R2 : 0.695517, test_R2 : 0.440341  train_MSE : 0.275900, test MSE : 0.373776
KNN - train_R2 : 0.438472, test_R2 : 0.212184  train_MSE : 0.374676, test MSE : 0.443467
SVR - train_R2 : 0.955434, test_R2 : 0.425191  train_MSE : 0.105553, test MSE : 0.378801
RF - train_R2 : 0.903345, test_R2 : 0.347970  train_MSE : 0.155447, test MSE : 0.403444
XGB - train_R2 : 0.638755, test_R2 : 0.357901  train_MSE : 0.300518, test MSE : 0.400359
